# Gray code encoding for ab initio interaction

In [3]:
import numpy as np
np.warnings.filterwarnings('ignore')

import pickle

from scipy.linalg import expm
# import scipy
from pprint import pprint
from tqdm import tqdm

from scipy.linalg import lstsq
import scipy as sp

# Everything we need from Qiskit
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute, Aer
from qiskit.quantum_info import Pauli

from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.components.initial_states import Custom

import qutip as qt
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set(rc={'figure.figsize':(16,8)})
sns.set()

import itertools

import sys
sys.path.append("../src/")
from hamiltonian import *
from utils import *
from qiskit_circuits import *
from qiskit_experiment import *

## Playing with Pauli and WeightedPauli operators

In [4]:
## H is always in qiskit order (right to left)
## String form is also in that order 
## State vector results in order [00,01,10,11]
from scipy.linalg import eigh

N_states=2
H_dense = DenseEncodingHamiltonian(N_states,qiskit_order=True,kill_bad_states=False)
print("pauli rep")
print(H_dense.pauli_rep)

thetas=[1.0]
backend=Aer.get_backend("statevector_simulator")
energy=compute_energy(thetas, backend, H_dense)
print("")
print(energy)

pauli rep
5.906709445000001 [] +
-4.286607049870561 [X0] +
-6.343290554999999 [Z0]

-1.1276405243971444


In [12]:
def compute_energy(theta, backend, hamiltonian, device=None, noise_model=None, shots=10000, num_cnot_pairs=0, num_folding=0, zero_noise_extrapolation=False):
    """
    Computes the expectation value of the hamiltonian by running a quantum circuit for
    wavefunction parameterized by theta.

    Parameters
        theta (np.array) : Initial guesses for the ground state wavefunction parameterization.

        backend (BaseBackend): Qiskit backend to execute circuits on.

        hamiltonian : Deuteron Hamiltonian. See src/hamiltonian.py for class definitions.

        device (Device, optional) : Simulations of IBMQ devices.  If device=None, qiskit QASM simulator
            applied without any noise model. Default: None.

        shots (int, optional): Number of repetitions of each circuit, for sampling. Default: 10000.


    """
    if( zero_noise_extrapolation and backend.name() == 'qasm_simulator' ):
        return compute_energy_extrap( theta, backend, hamiltonian, device, noise_model, shots, num_cnot_pairs, num_folding )
    # Cumulative energy
    energy = 0
    # Store the expectation values of all the operators
    complete_exp_vals = {}

    for measurement_idx in hamiltonian.pauli_partitions.keys():
        # Create the circuit depending on the Hamiltonian and measurement
        circuit = None
        if type(hamiltonian) == SparseEncodingHamiltonian:
            circuit = sparse_variational_circuit(theta, measurement_idx, backend.name(), num_cnot_pairs, num_folding)
        elif type(hamiltonian) == DenseEncodingHamiltonian:
            if measurement_idx == hamiltonian.N_qubits:
                circuit = dense_variational_circuit(theta, None, backend.name(), num_cnot_pairs, num_folding)
            else:
                circuit = dense_variational_circuit(theta, measurement_idx, backend.name(), num_cnot_pairs, num_folding)

        # Execute the circuit (on device if specified) and get the result
        # For QASM simulator, run the circuit for as many shots as desired.
#         print(hamiltonian.pauli_partitions[measurement_idx])
#         print(circuit.draw())
        if backend.name() == 'qasm_simulator':
            if( noise_model != None ):
                job = execute(circuit,
                        backend=backend,
                        shots = shots,
                        noise_model = noise_model,
                        basis_gates = noise_model.basis_gates,
                        optimization_level=0)
            elif device is not None:
                job = execute(circuit,
                        backend=backend,
                        shots = shots,
                        coupling_map = device.coupling_map,
                        noise_model = device.noise_model,
                        basis_gates = device.noise_model.basis_gates,
                        initial_layout = device.layout,
                        optimization_level=0)
            else:
                job = execute(circuit, backend, shots=shots, optimization_level=0)

            result = job.result().get_counts(circuit)

            # Perform measurement error mitigation if specified
            if device is not None:
                if device.meas_filter:
                    result = device.meas_filter.apply(result)

            # Get the expectation value of each Pauli from the results
            for pauli in hamiltonian.pauli_partitions[measurement_idx].keys():
                complete_exp_vals[pauli] = pauli_expectation_value(pauli, result)

        # For statevector simulator, compute the explicit wavefunction at the end
        # and calculate the expectation values using the trace.
        elif backend.name() == 'statevector_simulator':
            result = execute(circuit, backend, shots=1).result()
            psi = result.get_statevector(circuit)
            rho = np.einsum("i,j->ij", psi, np.conj(psi))

            # Paulis in the Hamiltonian are already in Qiskit ordering, which is the
            # same ordering as the psi that will come out.
            for pauli in hamiltonian.pauli_partitions[measurement_idx].keys():
                pauli_mat = get_pauli_matrix(pauli)
                complete_exp_vals[pauli] = np.real(np.trace(np.dot(rho, pauli_mat)))

    # Now that we have the complete set of expectation values, we can compute the energy
    for pauli, coeff in hamiltonian.pauli_coeffs.items():
        energy += coeff * complete_exp_vals[pauli]

    return energy

In [13]:
def do_experiment(theta,hamiltonian,parameters):
    """
    Use variational quantum eigensovler (VQE) to obtain ground state energy of the Hamiltonian.
    Energies at each step evaluated by running a quatnum circuit.

    Parameters
        theta (np.array) : Initial guesses for the ground state wavefunction theta parameters.

        hamiltonian : Deuteron Hamiltonian. See src/hamiltonian.py for class definitions.

        parameters (dictionary) : Input parameters for VQE.

    Returns
        res (scipy.optimize.OptimizeResult) : Results of VQE

    """
    # Set up backend
    backend = Aer.get_backend(parameters['backend'])
    device = None
    noise_model = None
    if parameters['device_name'] is not None:
        if ( parameters['device_name'][0:6] == "custom" ):
            noise_model = noise.NoiseModel()
            errors = parameters['device_name'].split("_")
            error1 = float(errors[1])
            error2 = float(errors[2])
            error_1 = noise.errors.depolarizing_error(error1, 1)
            error_2 = noise.errors.depolarizing_error(error2, 2)
            noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
            noise_model.add_all_qubit_quantum_error(error_2, ['cx'])
        else:
            if parameters['mitigate_meas_error']:
                device = Device(parameters['device_name'], True, hamiltonian.N_qubits, layout=parameters['layout'])
            else:
                device = Device(parameters['device_name'], False, hamiltonian.N_qubits, layout=parameters['layout'])

    if device is not None:
        print("Device specifications")
        pprint(vars(device))

    number_cnot_pairs = 0
    if( "number_cnot_pairs" in parameters ): number_cnot_pairs = parameters['number_cnot_pairs']
    number_circuit_folding = 0
    if( "number_circuit_folding" in parameters ): number_cnot_pairs = parameters['number_circuit_folding']
    zero_noise_extrapolation=False
    if( "zero_noise_extrapolation" in parameters): zero_noise_extrapolation = parameters['zero_noise_extrapolation']

    # Run the minimization routine
    if parameters['optimizer'] != 'SPSA':
        if parameters['optimizer'] == 'MIGRAD': # iminuit option
            res = iminimize(compute_energy, theta,
                    args=(backend, hamiltonian,
                        device,
                        noise_model,
                        parameters['N_shots'],
                        number_cnot_pairs,
                        number_circuit_folding,
                        zero_noise_extrapolation),
                    method=parameters['optimizer'], options={'maxfev':parameters['N_iter']})
            # Display diagnostic information:
            # res = iminimize(compute_energy, theta, args=(backend, hamiltonian, device), method=parameters['optimizer'], options={'maxfev':parameters['N_iter'],'disp':True})
        else:
            # "Regular" scipy optimizers
   
            res = minimize(compute_energy, theta,
                    args=(backend, hamiltonian,
                        device,
                        noise_model,
                        parameters['N_shots'],
                        number_cnot_pairs,
                        number_circuit_folding,
                        zero_noise_extrapolation),
                    method=parameters['optimizer'], options={'maxiter':parameters['N_iter']})
    else:
        res = minimizeSPSA(compute_energy,
                x0=theta,
                args=(backend, hamiltonian,
                    device,
                    noise_model,
                    parameters['N_shots'],
                    number_cnot_pairs,
                    number_circuit_folding,
                    zero_noise_extrapolation),
                niter=parameters['N_iter'],
                paired=False,
                a=parameters['spsa_a'],
                c=parameters['spsa_c'],
        )

    return res


In [18]:

parameters = {
"encoding": 'gray_code',
# "encoding": 'jordan_wigner',
"N_states": 3,
# "backend": 'statevector_simulator',
"backend": 'qasm_simulator',
"N_shots": 1000,
"N_cpus": 1,
"spsa_a": 0.628,
"spsa_c": 0.1,
"N_iter": 30,
"optimizer": 'SPSA',
"N_trials": N_trials,
"device_name": None,
"mitigate_meas_error": True,
"number_cnot_pairs": 4,
"zero_noise_extrapolation":True,
"output_dir": 'outputs-qiskit-statevector_simulator'
}


N_states=3
H_dense = DenseEncodingHamiltonian(N_states,qiskit_order=True,kill_bad_states=False)
print("pauli rep")
print(H_dense.pauli_rep)
thetas=[1.0,1.0]
# backend=Aer.get_backend("statevector_simulator")
backend=Aer.get_backend("qasm_simulator")
# energy=compute_energy(thetas, backend, H_dense)
# print("")
# print(energy)
res=do_experiment(thetas,H_dense,parameters)
print(res)

pauli rep
7.7658547225 [] +
-2.1433035249352805 [X0] +
-2.1433035249352805 [X0 Z1] +
-7.9841452775 [Z0] +
3.913118960624632 [Z0 X1] +
1.6408547225000003 [Z0 Z1] +
-3.913118960624632 [X1] +
-1.8591452774999997 [Z1]
     fun: -2.043234965306945
 message: 'terminated after reaching max number of iterations'
    nfev: 60
     nit: 30
 success: True
       x: array([0.54872 , 0.801501])


In [15]:
N_states=2
H_sparse = SparseEncodingHamiltonian(N_states,qiskit_order=True)
print("pauli rep")
print(H_sparse.pauli_rep)

thetas=[1.0]
backend=Aer.get_backend("statevector_simulator")
energy=compute_energy(thetas, backend, H_sparse)
print("")
print(energy)

pauli rep
5.906709445000001 [] +
-2.1433035249352805 [X0 X1] +
-2.1433035249352805 [Y0 Y1] +
0.21829055499999983 [Z0] +
-6.125 [Z1]

5.72694850291904


In [17]:
N_states=3
N_trials=10
thetas=[[1.0]*(N_states-1)]*N_trials

parameters = {
"encoding": 'jordan_wigner',
"N_states": N_states,
# "backend": 'statevector_simulator',
"backend": 'qasm_simulator',
"N_shots": 10000,
"N_cpus": 1,
"spsa_a": 0.628,
"spsa_c": 0.1,
"N_iter": 1000,
"optimizer": 'SPSA',
"N_trials": N_trials ,
"device_name": None,
"mitigate_meas_error": False,
"output_dir": 'outputs-qiskit-statevector_simulator'
}

H_sparse = SparseEncodingHamiltonian(N_states,qiskit_order=True)
print(H_sparse.pauli_rep)

energies=[]
for theta in thetas:
    res=do_experiment(theta,H_sparse,parameters)
    energies.append(res["fun"])
print(energies)

15.531709445 [] +
-2.1433035249352805 [X0 X1] +
-2.1433035249352805 [Y0 Y1] +
0.21829055499999983 [Z0] +
-3.913118960624632 [X1 X2] +
-3.913118960624632 [Y1 Y2] +
-6.125 [Z1] +
-9.625 [Z2]
[-1.984317344863853, -1.8993488056642667, -2.2000654643973006, -2.046516616945258, -2.0986968064528257, -1.9120860729031879, -2.181177595814831, -2.0472582301450855, -1.9150290254000408, -2.0697366887843804]
